In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install openmim for installation of mmcv-full
!pip install openmim

# Install mmcv-full thus we could use CUDA operators
!mim install mmcv-full

# Clone MMEditing
!rm -rf mmediting
!git clone https://github.com/open-mmlab/mmediting.git
%cd mmediting

# Install MMEditing
!pip install -v -e .

mmcv-full==1.4.8 existed.
Cloning into 'mmediting'...
remote: Enumerating objects: 10451, done.
remote: Counting objects: 100% (1124/1124), done.
remote: Compressing objects: 100% (719/719), done.
remote: Total 10451 (delta 561), reused 757 (delta 382), pack-reused 9327
Receiving objects: 100% (10451/10451), 6.65 MiB | 20.27 MiB/s, done.
Resolving deltas: 100% (6872/6872), done.
/content/mmediting
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/lo

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMEditing installation
import mmedit
print(mmedit.__version__)

1.10.0+cu111 True
0.14.0


In [ ]:
'''import glob


gt_paths = sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/AI6126-Project2/train/GT2/*.png'))
with open('/content/drive/MyDrive/Colab Notebooks/AI6126-Project2/training_ann2.txt', 'w') as f:
  for gt_path in gt_paths:
    filename = gt_path.split('/')[-1]
    line = f'{filename} (512,512,3)\n'
    f.write(line)

# Let's make sure the annotations are correct
with open('/content/drive/MyDrive/Colab Notebooks/AI6126-Project2/training_ann2.txt', 'r') as f:
  file_content = f.read()
  print(file_content)'''

"import glob\n\n\ngt_paths = sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/AI6126-Project2/train/GT2/*.png'))\nwith open('/content/drive/MyDrive/Colab Notebooks/AI6126-Project2/training_ann2.txt', 'w') as f:\n  for gt_path in gt_paths:\n    filename = gt_path.split('/')[-1]\n    line = f'{filename} (512,512,3)\n'\n    f.write(line)\n\n# Let's make sure the annotations are correct\nwith open('/content/drive/MyDrive/Colab Notebooks/AI6126-Project2/training_ann2.txt', 'r') as f:\n  file_content = f.read()\n  print(file_content)"

##Modify The Config File

In [ ]:
from mmcv import Config
# Load the original config
cfg = Config.fromfile('/content/drive/MyDrive/srresnet_ffhq_300k.py')

#print(f'Config:\n{cfg.pretty_text}')  # Show the config

Given a configuration that trains a model on the whole DIV2K, we need to modify the configuration so that it trains on our tiny dataset.

In [ ]:
from mmcv.runner import set_random_seed

# Training folders
cfg.data.train.dataset.lq_folder = '/content/drive/MyDrive/data/train/GT'
cfg.data.train.dataset.gt_folder = '/content/drive/MyDrive/data/train/GT'
#cfg.data.train.dataset.ann_file = '/content/drive/MyDrive/Colab Notebooks/AI6126-Project2/training_ann.txt'

# Validation folders
cfg.data.val.lq_folder = '/content/drive/MyDrive/data/val/LQ'
cfg.data.val.gt_folder = '/content/drive/MyDrive/data/val/GT'


# Set up working dir to save files and logs
cfg.work_dir = '/content/drive/MyDrive/work_dir_srgan'


# Set seed thus the results are reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpus = 1

##Training

In [ ]:
import os.path as osp

from mmedit.datasets import build_dataset
from mmedit.models import build_model
from mmedit.apis import train_model
from mmcv.runner import init_dist

import mmcv
import os

# Initialize distributed training (only need to initialize once), comment it if
# have already run this part
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '50297'
init_dist('pytorch', **cfg.dist_params)


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the SRCNN model
model = build_model(
        cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

# Meta information
meta = dict()
if cfg.get('exp_name', None) is None:
    cfg['exp_name'] = osp.splitext(osp.basename(cfg.work_dir))[0]
meta['exp_name'] = cfg.exp_name
meta['mmedit Version'] = mmedit.__version__
meta['seed'] = 0

# Train the model
train_model(model, datasets, cfg, distributed=True, validate=True, meta=meta)


##Testing

In [ ]:
import cv2
import matplotlib.pyplot as plt
import mmcv
import torch
import torchvision

from mmedit.models import build_model
from mmcv.runner import load_checkpoint

from mmedit.apis import single_gpu_test
from mmedit.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

from mmedit.apis import init_model, restoration_inference

from mmedit.core import tensor2img


# Build a test dataloader and model
config = '/content/drive/MyDrive/srresnet_ffhq_300k.py'
checkpoint = '/content/drive/MyDrive/iter_300000.pth'

model = init_model(
        config, checkpoint, device=torch.device('cuda'))

for i in range (0, 400):
  img_path = "/content/drive/MyDrive/data/test/LQ/"+ "%05d" % i + '.png'
  print(img_path)

  output = restoration_inference(model, img_path)
  output = tensor2img(output)

  save_path = "/content/drive/MyDrive/data/test/LQ/results/"+ "%05d" % i + '.png'

  mmcv.imwrite(output, save_path)

  i= i+1